In [469]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import subgraph, k_hop_subgraph, dense_to_sparse
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score
import torch.nn.functional as FF

In [470]:
def load_dataset_Cora():
    dataset=Planetoid(root='tmp/Cora', name='Cora')
    return dataset

In [471]:
def load_dataset_PubMed():
    dataset=Planetoid(root='tmp/PubMed', name='PubMed')
    return dataset

In [472]:
class GCN_Extraction(nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN_Extraction,self).__init__()
        self.conv1=GCNConv(num_features,16)
        self.conv2=GCNConv(16, num_classes)
        
    def forward(self, x, edge_index):
        x=self.conv1(x,edge_index)
        x=torch.relu(x)
        x=self.conv2(x,edge_index)
        return torch.log_softmax(x, dim=1)

In [473]:
class GCN_Victim(nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN_Victim,self).__init__()
        self.conv1=GCNConv(num_features,128)
        self.conv2=GCNConv(128,64)
        self.conv3=GCNConv(64, 16)
        self.conv4=GCNConv(16,num_classes)

    def forward(self, x, edge_index):
        x=self.conv1(x,edge_index)
        x=torch.relu(x)
        x = self.conv2(x, edge_index)
        x=torch.relu(x)
        x=self.conv3(x,edge_index)
        x=torch.relu(x)
        x=self.conv4(x, edge_index)
        return torch.log_softmax(x, dim=1)

In [474]:
def train_Victim(model, data, epoches=300, lr=0.01, weight_decay=5e-4):
    optimizer=optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epoches):
        optimizer.zero_grad()
        out=model(data.x, data.edge_index)
        loss=FF.nll_loss(out[data.train_mask],data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 10 ==0:
            print(f"Epoch {epoch+10}, Loss: {loss.item():.5f}")

In [475]:
def train_Extraction(model,features, adjacency_matrix, labels, epoches=400, lr=0.01, weight_decay=5e-4):
    optimizer=optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
    model.train()

    patience_counter = 0
    
    for epoch in range(epoches):
        optimizer.zero_grad()
        out=model(features, adjacency_matrix)
        loss=FF.nll_loss(out,labels)
        loss.backward()
        optimizer.step()
            
        if epoch % 10 ==0:
            print(f"Epoch {epoch+10}, Loss: {loss.item():.5f}")

In [476]:
# Function to extract a 2-hop subgraph from a larger graph.
# It samples a random node and retrieves its 2-hop neighborhood if its size is within the specified limits.
def get_2hop_subgraph(data, min_size=10, max_size=150):
    num_nodes=data.num_nodes

    while True:
        node_idx=torch.randint(0,num_nodes,(1,)).item()
        
        subset, edge_index,_,_=k_hop_subgraph(node_idx, 2, data.edge_index, relabel_nodes=True)

        if min_size<=subset.size(0)<=max_size:
            As=torch.zeros((subset.size(0),subset.size(0)))
            As[edge_index[0],edge_index[1]]=1

            Xs=data.x[subset]

            # Return the adjacency matrix, features, index of the center node, the mapping relationship between the index and the node
            return As.numpy(), Xs.numpy(), node_idx, subset

In [477]:
# Function to compute the prior distributions for both feature occurrence and feature count
def get_distribution(num_classes):
    F=[] # List to store feature distribution for each class
    M=[] # List to store feature count distribution for each class
    
    for c in range(num_classes):
        class_nodes=data.x[labels==c]

        feature_counts=class_nodes.sum(dim=0).numpy()
        feature_distribution=feature_counts/feature_counts.sum()
        F.append(feature_distribution)

        num_features_per_node=class_nodes.sum(dim=1).numpy()
        feature_count_distribution=np.bincount(num_features_per_node.astype(int),minlength=num_features)
        M.append(feature_count_distribution/feature_count_distribution.sum())
        
    return F,M

In [478]:
# Function to generate a sample graph based on input feature (Fc) and count (Mc) distributions 
def GenerateSample(Fc, Mc, As):
    num_nodes=As.shape[0]
    Ac=torch.ones((num_nodes,num_nodes))
    Xc=torch.zeros(num_nodes, len(Fc))

    for i in range(num_nodes):
        m=np.random.choice(np.arange(len(Mc)),p=Mc)
        features=np.random.choice(len(Fc),size=m,replace=False,p=Fc)
        Xc[i,features]=1

    return Ac.numpy(), Xc.numpy()

In [479]:
# Function to sample subgraphs and combine them into a larger graph
def SubgraphSamplingAlgorithm(Gs,F, M, n, C):
    As, Xs=Gs
    num_nodes=As.shape[0]
    SA=[As]
    SX=[Xs]
    SL=[api_out_class(Xs,As)]

    for i in range(n):
        # For each class, generate and store a new sampled subgraph
        for c in range(C):
            Ac, Xc=GenerateSample(F[c],M[c],As)
            SA.append(Ac)
            SX.append(Xc)
            SL.append(api_out_class(Xc,Ac))

    AG_list=[dense_to_sparse(torch.tensor(a))[0] for a in SA]
    XG=torch.vstack([torch.tensor(x) for x in SX])

    SL=torch.tensor(SL,dtype=torch.long).view(-1)

    # Filter out invalid labels (negative labels) and trim the labels to match the feature matrix size
    valid_mask = SL >= 0
    SL = SL[valid_mask]
    SL = SL[:XG.shape[0]]

    # Combine the edge indices of all subgraphs, adjusting the node indices to avoid overlap
    AG_combined=torch.cat([edge_index+i*num_nodes for i, edge_index in enumerate (AG_list)], dim=1)

    return XG, AG_combined, SL

In [480]:
# Function to predict class labels for a graph using its feature matrix and adjacency matrix.
def api_out_class(features, adjacency_matrix):
    features_tensor=torch.tensor(features,dtype=torch.float)
    adjacency_tensor=torch.tensor(adjacency_matrix,dtype=torch.long)

    # If the adjacency matrix is in dense form, convert it to edge list format (sparse)
    if adjacency_tensor.ndim==2 and adjacency_tensor.shape[0]==adjacency_tensor.shape[1]:
        adjacency_tensor=torch.nonzero(adjacency_tensor, as_tuple=False).t()

    with torch.no_grad():
        logits=f(features_tensor, adjacency_tensor)
        predictions = torch.argmax(logits, dim=1)

    
    return predictions.numpy()

In [481]:
# Function to compute and return the raw logits (class scores) for a graph using its feature matrix and adjacency matrix.
def api_out_vector(features, adjacency_matrix):
    features_tensor=torch.tensor(features,dtype=torch.float)
    adjacency_tensor=torch.tensor(adjacency_matrix,dtype=torch.long)

    # If the adjacency matrix is in dense form (square matrix), convert it to a sparse edge list format
    if adjacency_tensor.ndim==2 and adjacency_tensor.shape[0]==adjacency_tensor.shape[1]:
        adjacency_tensor=torch.nonzero(adjacency_tensor, as_tuple=False).t()
        
    with torch.no_grad():
        logits=f(features_tensor, adjacency_tensor)

    return logits.numpy()

In [482]:
# Function to evaluate a model and return predictions for nodes based on a mask.
def api(model, data, mask):
    model.eval()
    out=model(data.x, data.edge_index)
    pred=out[mask].argmax(dim=1)

    return pred

In [483]:
# Function to evaluate how well the extracted model replicates the predictions of the victim model.
def eval(victim_model, extracted_model,data, mask):
    original_preds_labels=api(victim_model,data,mask)
    extracted_preds_labels=api(extracted_model,data,mask)
    score=accuracy_score(original_preds_labels, extracted_preds_labels)

    return score

In [484]:
# Function to evaluate the extraction model multiple times and return the average accuracy score.
def get_f(victim_model,extraction_model,data,mask,times):
    list_out=[]
    for i in range(times):
        pred=eval(victim_model,extraction_model,data,mask)
        list_out.append(pred)
    
    return sum(list_out)/len(list_out)

In [485]:
# Function to find the index of the center node in the subgraph.
def find_center_node_in_subgraph(center_global_idx, node_mapping):
    center_subgraph_idx = node_mapping.tolist().index(center_global_idx)
    return center_subgraph_idx

In [486]:
# Function to approximate new nodes in the graph Gc based on KL divergence between nodes and the center node.
def approximate_nodes(Gc, api_out_vector, F, M, C, rho):
    A,X=Gc
    L=api_out_vector(X, A)
    center_idx=find_center_node_in_subgraph(center_node,node_mapping)
    
    for node in range(A.shape[0]):
        if node!=center_idx:
            for c in range(C):
                kl_div = np.exp(L[node][c]) * (L[node][c] - L[center_idx][c])
                
                if kl_div > 0:
                    num_new_nodes = int(kl_div * rho * np.log1p(A[node].sum())) # TODO
                    for _ in range(num_new_nodes):
                        new_node = A.shape[0]
                        
                        new_adj_row = np.zeros((1, A.shape[1]))
                        A = np.vstack([A, new_adj_row])
                        
                        new_adj_col = np.zeros((A.shape[0], 1))
                        A = np.hstack([A, new_adj_col])
                        
                        A[node, new_node] = 1
                        A[new_node, node] = 1
                        
                        feature_count = np.random.choice(len(M[c]), p=M[c])  
                        new_features = np.zeros_like(F[c])
                        chosen_features = np.random.choice(len(F[c]), size=feature_count, p=F[c])  
                        new_features[chosen_features] = 1 
                        
                        X = np.vstack([X, new_features])
    
    Gc=(A,X)
    return Gc

In [487]:
dataset=load_dataset_PubMed()

/opt/anaconda3/envs/gnn/lib/python3.9/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load(f) != 

In [488]:
data=dataset[0]

In [489]:
labels=data.y.numpy()

In [490]:
num_classes=dataset.num_classes

In [491]:
num_features=data.x.shape[1]

In [492]:
F,M=get_distribution(num_classes)

In [493]:
runturns=100

In [494]:
f=GCN_Victim(data.x.shape[1],dataset.num_classes)

In [495]:
train_Victim(f,data)

Epoch 10, Loss: 1.09910
Epoch 20, Loss: 0.63681
Epoch 30, Loss: 0.13503
Epoch 40, Loss: 0.01758
Epoch 50, Loss: 0.00479
Epoch 60, Loss: 0.00709
Epoch 70, Loss: 0.00985
Epoch 80, Loss: 0.00707
Epoch 90, Loss: 0.00484
Epoch 100, Loss: 0.00413
Epoch 110, Loss: 0.00389
Epoch 120, Loss: 0.00350
Epoch 130, Loss: 0.00340
Epoch 140, Loss: 0.00328
Epoch 150, Loss: 0.00317
Epoch 160, Loss: 0.00307
Epoch 170, Loss: 0.00305
Epoch 180, Loss: 0.00301
Epoch 190, Loss: 0.00306
Epoch 200, Loss: 0.00299
Epoch 210, Loss: 0.00313
Epoch 220, Loss: 0.00275
Epoch 230, Loss: 0.00295
Epoch 240, Loss: 0.74381
Epoch 250, Loss: 0.54549
Epoch 260, Loss: 0.17659
Epoch 270, Loss: 0.04379
Epoch 280, Loss: 0.01047
Epoch 290, Loss: 0.00495
Epoch 300, Loss: 0.00341


In [496]:
As, Xs, center_node, node_mapping=get_2hop_subgraph(data)

In [497]:
Gs=(As,Xs)

In [498]:
print(f"The index of the node is {center_node}")

The index of the node is 17907


In [499]:
XG, AG, SL=SubgraphSamplingAlgorithm(Gs,F, M, n=10, C=dataset.num_classes)

In [500]:
g=GCN_Extraction(XG.shape[1], dataset.num_classes)

In [501]:
train_Extraction(g,XG,AG,SL)

Epoch 10, Loss: 1.10024
Epoch 20, Loss: 0.85853
Epoch 30, Loss: 0.48853
Epoch 40, Loss: 0.21327
Epoch 50, Loss: 0.09546
Epoch 60, Loss: 0.05684
Epoch 70, Loss: 0.04568
Epoch 80, Loss: 0.04340
Epoch 90, Loss: 0.04294
Epoch 100, Loss: 0.04163
Epoch 110, Loss: 0.03937
Epoch 120, Loss: 0.03687
Epoch 130, Loss: 0.03465
Epoch 140, Loss: 0.03284
Epoch 150, Loss: 0.03135
Epoch 160, Loss: 0.03007
Epoch 170, Loss: 0.02890
Epoch 180, Loss: 0.02784
Epoch 190, Loss: 0.02689
Epoch 200, Loss: 0.02603
Epoch 210, Loss: 0.02525
Epoch 220, Loss: 0.02453
Epoch 230, Loss: 0.02387
Epoch 240, Loss: 0.02327
Epoch 250, Loss: 0.02271
Epoch 260, Loss: 0.02220
Epoch 270, Loss: 0.02172
Epoch 280, Loss: 0.02127
Epoch 290, Loss: 0.02085
Epoch 300, Loss: 0.02046
Epoch 310, Loss: 0.02010
Epoch 320, Loss: 0.01975
Epoch 330, Loss: 0.01942
Epoch 340, Loss: 0.01912
Epoch 350, Loss: 0.01884
Epoch 360, Loss: 0.01856
Epoch 370, Loss: 0.01831
Epoch 380, Loss: 0.01807
Epoch 390, Loss: 0.01784
Epoch 400, Loss: 0.01763


In [502]:
print(get_f(f,g,data,data.val_mask,runturns))

0.6940000000000011


In [503]:
# For Algorithm 2

In [504]:
Gs_with_approximated_nodes = approximate_nodes(Gs, api_out_vector, F, M, C=dataset.num_classes,rho=2)

In [505]:
print(f"The size of tensors with algo1{Gs[0].shape,Gs[1].shape}")
print(f"The size of tensors with algo2{Gs_with_approximated_nodes[0].shape,Gs_with_approximated_nodes[1].shape}")

The size of tensors with algo1((24, 24), (24, 500))
The size of tensors with algo2((24, 24), (24, 500))


In [506]:
XG1, AG1, SL1=SubgraphSamplingAlgorithm(Gs_with_approximated_nodes,F, M, n=10, C=dataset.num_classes)

In [507]:
g1=GCN_Extraction(XG1.shape[1], dataset.num_classes)

In [508]:
train_Extraction(g1,XG1,AG1,SL1)

Epoch 10, Loss: 1.09922
Epoch 20, Loss: 0.80643
Epoch 30, Loss: 0.46236
Epoch 40, Loss: 0.21117
Epoch 50, Loss: 0.09623
Epoch 60, Loss: 0.05660
Epoch 70, Loss: 0.04386
Epoch 80, Loss: 0.04056
Epoch 90, Loss: 0.03990
Epoch 100, Loss: 0.03889
Epoch 110, Loss: 0.03703
Epoch 120, Loss: 0.03481
Epoch 130, Loss: 0.03269
Epoch 140, Loss: 0.03089
Epoch 150, Loss: 0.02944
Epoch 160, Loss: 0.02820
Epoch 170, Loss: 0.02708
Epoch 180, Loss: 0.02609
Epoch 190, Loss: 0.02517
Epoch 200, Loss: 0.02435
Epoch 210, Loss: 0.02359
Epoch 220, Loss: 0.02292
Epoch 230, Loss: 0.02228
Epoch 240, Loss: 0.02171
Epoch 250, Loss: 0.02116
Epoch 260, Loss: 0.02070
Epoch 270, Loss: 0.02025
Epoch 280, Loss: 0.01984
Epoch 290, Loss: 0.01943
Epoch 300, Loss: 0.01905
Epoch 310, Loss: 0.01872
Epoch 320, Loss: 0.01841
Epoch 330, Loss: 0.01811
Epoch 340, Loss: 0.01783
Epoch 350, Loss: 0.01757
Epoch 360, Loss: 0.01732
Epoch 370, Loss: 0.01709
Epoch 380, Loss: 0.01687
Epoch 390, Loss: 0.01666
Epoch 400, Loss: 0.01647


In [509]:
print(get_f(f,g1,data,data.val_mask,runturns))

0.6300000000000011
